In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import time
import imp
import os
import os, os.path
import datetime as dt
import dateutil.parser as dp
import dill

%matplotlib inline

In [25]:
# Create lists of data files (3 groups: regular attributes, GEO, and Seller composition)

data_directory = "../30y fixed rate data/fnm/conforming"

# pools attributes 
pools_attributes_files = list()

for root, dirs, files in os.walk(data_directory):
    for name in files:
        file_name = os.path.join(root, name)
        if 'pools attributes' in file_name:
            pools_attributes_files.append(file_name)
        
# pools geographical composition        
pools_geo_pct_files = list()

for root, dirs, files in os.walk(data_directory):
    for name in files:
        file_name = os.path.join(root, name)
        if 'geo pct' in file_name:
            pools_geo_pct_files.append(os.path.join(root, name))  
        
# pools seller composition        
pools_seller_pct_files = list()

for root, dirs, files in os.walk(data_directory):
    for name in files:
        file_name = os.path.join(root, name)
        if 'seller pct' in file_name:
            pools_seller_pct_files.append(os.path.join(root, name)) 
        
print("# of attr files: ",len(pools_attributes_files))
print("# of geo pct files: ",len(pools_geo_pct_files))
print("# of seller pct files: ",len(pools_seller_pct_files))

# of attr files:  251
# of geo pct files:  250
# of seller pct files:  252


In [27]:
# Read in all files with regular pool attributes
attr_df = pd.DataFrame()
for file in pools_attributes_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(attr_df)==0:
        attr_df = one_month_issue_pools
    else:
        # df1.columns.difference(df2.columns)
        # df2.columns.difference(df1.columns)
        if all(attr_df.columns == one_month_issue_pools.columns):
            attr_df = pd.concat([attr_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')

    #print('Done loading ' + file)

C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result

In [28]:
attr_df.columns

Index(['poolno', 'asofdate', 'cusip', 'Prefix', 'spread', 'Cpr1', 'Cpr3',
       'Cpr6', 'Cpr12', 'Cpr24', 'CprLife', 'SMM', 'DayCount', 'OBal', 'CBal',
       'pbal', 'paydown', 'Prepay', 'factor', 'OCoupon', 'Coupon', 'OWac',
       'Wac', 'Wam', 'Age', 'aols', 'waols', 'ONLoans', 'cnloans', 'PCNLoans',
       'PPNLoans', 'OSato', 'CSato', 'oltv', 'cltv', '%cltv_80', '%cltv_105',
       '%cltv_125', '%ccltv_80', '%ccltv_105', '%ccltv_125', 'fico',
       '%FedHold', '%CMOHold', 'dti', 'codti', '%CashWindow', '%Majors',
       'ocltv', 'ccltv', 'PurpPct_purchase', 'PurpPct_refi',
       'PctChannel_Broker', 'PctChannel_Corr', 'PctChannel_Retail',
       'OccPct_investor', 'OccPct_owner', 'PropUnitsPct_2-4', 'Label'],
      dtype='object')

In [29]:
print(f'attr_df.shape = {attr_df.shape}')

attr_df.shape = (12721779, 59)


In [30]:
# Read in all files with pools GEO info
geo_df = pd.DataFrame()
for file in pools_geo_pct_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(geo_df)==0:
        geo_df = one_month_issue_pools
    else:
        # df1.columns.difference(df2.columns)
        # df2.columns.difference(df1.columns)
        if all(geo_df.columns == one_month_issue_pools.columns):
            geo_df = pd.concat([geo_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')

    #print('Done loading ' + file)

C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, re

In [31]:
geo_df.columns

Index(['poolno', 'asofdate', 'cusip', 'StatePct_AK', 'StatePct_AL',
       'StatePct_AR', 'StatePct_AZ', 'StatePct_CA', 'StatePct_CO',
       'StatePct_CT', 'StatePct_DC', 'StatePct_DE', 'StatePct_FL',
       'StatePct_GA', 'StatePct_GU', 'StatePct_HI', 'StatePct_IA',
       'StatePct_ID', 'StatePct_IL', 'StatePct_IN', 'StatePct_KS',
       'StatePct_KY', 'StatePct_LA', 'StatePct_MA', 'StatePct_MD',
       'StatePct_ME', 'StatePct_MI', 'StatePct_MN', 'StatePct_MO',
       'StatePct_MS', 'StatePct_MT', 'StatePct_NC', 'StatePct_ND',
       'StatePct_NE', 'StatePct_NH', 'StatePct_NJ', 'StatePct_NM',
       'StatePct_NV', 'StatePct_NY', 'StatePct_OH', 'StatePct_OK',
       'StatePct_OR', 'StatePct_PA', 'StatePct_PR', 'StatePct_RI',
       'StatePct_SC', 'StatePct_SD', 'StatePct_TN', 'StatePct_TX',
       'StatePct_UT', 'StatePct_VA', 'StatePct_VI', 'StatePct_VT',
       'StatePct_WA', 'StatePct_WI', 'StatePct_WV', 'StatePct_WY', 'Label'],
      dtype='object')

In [32]:
print(f'geo_df.shape = {geo_df.shape}')

geo_df.shape = (11040282, 58)


In [33]:
# Read in all files with pools Seller info
seller_df = pd.DataFrame()
for file in pools_seller_pct_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(seller_df)==0:
        seller_df = one_month_issue_pools
    else:
        # df1.columns.difference(df2.columns)
        # df2.columns.difference(df1.columns)
        if all(seller_df.columns == one_month_issue_pools.columns):
            seller_df = pd.concat([seller_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')
    #print('Done loading ' + file)

C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,13,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=comp

In [34]:
seller_df.columns

Index(['poolno', 'asofdate', 'SellerPct_AMRHT', 'SellerPct_ALS',
       'SellerPct_CAFULL', 'SellerPct_CNTL', 'SellerPct_CITIZ', 'SellerPct_53',
       'SellerPct_FIR', 'SellerPct_FRDOM', 'SellerPct_GUILD',
       'SellerPct_CHASE', 'SellerPct_LLSL', 'SellerPct_MATRX', 'SellerPct_NCM',
       'SellerPct_NATIONST', 'SellerPct_NRESM', 'SellerPct_PNYMAC',
       'SellerPct_PILOSI', 'SellerPct_QUICK', 'SellerPct_REG',
       'SellerPct_RMSC', 'SellerPct_UNSHFI', 'SellerPct_WFHM', 'cusip',
       'Prefix', 'Label'],
      dtype='object')

In [35]:
print(f'seller_df.shape = {seller_df.shape}')

seller_df.shape = (11123306, 27)


In [36]:
print(f'attr_df.shape = {attr_df.shape}')

attr_df.shape = (12721779, 59)


In [37]:
print(f'geo_df.shape = {geo_df.shape}')

geo_df.shape = (11040282, 58)


In [38]:
attr_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12721779 entries, 0 to 1657
Data columns (total 59 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   poolno             12721779 non-null  object 
 1   asofdate           12721779 non-null  int64  
 2   cusip              12721779 non-null  object 
 3   Prefix             12721779 non-null  object 
 4   spread             12716611 non-null  object 
 5   Cpr1               12717900 non-null  float64
 6   Cpr3               12374521 non-null  float64
 7   Cpr6               11866842 non-null  object 
 8   Cpr12              10887508 non-null  float64
 9   Cpr24              9073002 non-null   object 
 10  CprLife            12716605 non-null  float64
 11  SMM                12720854 non-null  float64
 12  DayCount           12721770 non-null  float64
 13  OBal               12721770 non-null  float64
 14  CBal               12716603 non-null  object 
 15  pbal             

In [39]:
attr_df.drop(['Prefix','Cpr1','Cpr3','Cpr6','Cpr12','Cpr24','CprLife','pbal','paydown','Prepay','OCoupon','OWac','PCNLoans','PPNLoans','OSato','%cltv_80','%cltv_105','%cltv_125','%ccltv_80','%ccltv_105','%ccltv_125','%FedHold','%CMOHold','dti','codti'],axis=1,inplace=True)

In [40]:
attr_df.dropna(subset=['CBal','SMM','Wac','aols','fico','OccPct_investor','OccPct_owner','PropUnitsPct_2-4'],inplace=True)

In [41]:
attr_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [42]:
attr_df.fillna(method='ffill',inplace=True)

In [43]:
attr_df['CBal'] = attr_df['CBal'].apply(lambda x: float(x))

In [44]:
attr_df.rename(columns={'Label':'pool_issue_month'},inplace=True)

In [45]:
attr_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12687671 entries, 1482 to 21251
Data columns (total 34 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   poolno             12687671 non-null  object 
 1   asofdate           12687671 non-null  int64  
 2   cusip              12687671 non-null  object 
 3   spread             12687671 non-null  object 
 4   SMM                12687671 non-null  float64
 5   DayCount           12687671 non-null  float64
 6   OBal               12687671 non-null  float64
 7   CBal               12687671 non-null  float64
 8   factor             12687671 non-null  float64
 9   Coupon             12687671 non-null  float64
 10  Wac                12687671 non-null  float64
 11  Wam                12687671 non-null  float64
 12  Age                12687671 non-null  float64
 13  aols               12687671 non-null  float64
 14  waols              12687671 non-null  object 
 15  ONLoans      

In [46]:
geo_df.drop('Label',axis=1,inplace=True)

In [47]:
geo_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [48]:
geo_df.fillna(method='ffill',inplace=True)

In [49]:
geo_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11040282 entries, 1482 to 4436
Data columns (total 57 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   poolno       11040282 non-null  object 
 1   asofdate     11040282 non-null  int64  
 2   cusip        11040282 non-null  object 
 3   StatePct_AK  11040282 non-null  float64
 4   StatePct_AL  11040282 non-null  object 
 5   StatePct_AR  11040282 non-null  float64
 6   StatePct_AZ  11040282 non-null  float64
 7   StatePct_CA  11040282 non-null  float64
 8   StatePct_CO  11040282 non-null  object 
 9   StatePct_CT  11040282 non-null  float64
 10  StatePct_DC  11040282 non-null  float64
 11  StatePct_DE  11040282 non-null  float64
 12  StatePct_FL  11040282 non-null  float64
 13  StatePct_GA  11040282 non-null  float64
 14  StatePct_GU  11040282 non-null  float64
 15  StatePct_HI  11040282 non-null  float64
 16  StatePct_IA  11040282 non-null  float64
 17  StatePct_ID  11040282 non-

In [50]:
geo_columns = ['StatePct_AK','StatePct_AL','StatePct_AR','StatePct_AZ','StatePct_CA','StatePct_CO','StatePct_CT',
                'StatePct_DC','StatePct_DE','StatePct_FL','StatePct_GA','StatePct_GU','StatePct_HI','StatePct_IA',
                'StatePct_ID','StatePct_IL','StatePct_IN','StatePct_KS','StatePct_KY','StatePct_LA','StatePct_MA',
                'StatePct_MD','StatePct_ME','StatePct_MI','StatePct_MN','StatePct_MO','StatePct_MS','StatePct_MT',
                'StatePct_NC','StatePct_ND','StatePct_NE','StatePct_NH','StatePct_NJ','StatePct_NM','StatePct_NV',
                'StatePct_NY','StatePct_OH','StatePct_OK','StatePct_OR','StatePct_PA','StatePct_PR','StatePct_RI',
                'StatePct_SC','StatePct_SD','StatePct_TN','StatePct_TX','StatePct_UT','StatePct_VA','StatePct_VI',
                'StatePct_VT','StatePct_WA','StatePct_WI','StatePct_WV','StatePct_WY']

In [51]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [52]:
geo_df[geo_columns] = geo_df[geo_columns].applymap(lambda x: float(x) if isfloat(x) else np.nan)

In [53]:
geo_df.fillna(0,inplace=True)

In [54]:
geo_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11040282 entries, 1482 to 4436
Data columns (total 57 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   poolno       11040282 non-null  object 
 1   asofdate     11040282 non-null  int64  
 2   cusip        11040282 non-null  object 
 3   StatePct_AK  11040282 non-null  float64
 4   StatePct_AL  11040282 non-null  float64
 5   StatePct_AR  11040282 non-null  float64
 6   StatePct_AZ  11040282 non-null  float64
 7   StatePct_CA  11040282 non-null  float64
 8   StatePct_CO  11040282 non-null  float64
 9   StatePct_CT  11040282 non-null  float64
 10  StatePct_DC  11040282 non-null  float64
 11  StatePct_DE  11040282 non-null  float64
 12  StatePct_FL  11040282 non-null  float64
 13  StatePct_GA  11040282 non-null  float64
 14  StatePct_GU  11040282 non-null  float64
 15  StatePct_HI  11040282 non-null  float64
 16  StatePct_IA  11040282 non-null  float64
 17  StatePct_ID  11040282 non-

In [55]:
seller_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11123306 entries, 0 to 1657
Data columns (total 27 columns):
 #   Column              Non-Null Count     Dtype  
---  ------              --------------     -----  
 0   poolno              11123306 non-null  object 
 1   asofdate            11123306 non-null  int64  
 2   SellerPct_AMRHT     11119432 non-null  object 
 3   SellerPct_ALS       11119427 non-null  float64
 4   SellerPct_CAFULL    11119427 non-null  float64
 5   SellerPct_CNTL      11119424 non-null  float64
 6   SellerPct_CITIZ     11119424 non-null  float64
 7   SellerPct_53        11119424 non-null  float64
 8   SellerPct_FIR       11119406 non-null  float64
 9   SellerPct_FRDOM     11119406 non-null  float64
 10  SellerPct_GUILD     11119406 non-null  object 
 11  SellerPct_CHASE     11119398 non-null  float64
 12  SellerPct_LLSL      11119398 non-null  float64
 13  SellerPct_MATRX     11119398 non-null  object 
 14  SellerPct_NCM       11119384 non-null  float64
 15  

In [56]:
seller_df.drop(['Label','Prefix'],axis=1,inplace=True)

In [57]:
seller_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [58]:
seller_df.fillna(method='ffill',inplace=True)

In [59]:
seller_columns = ['SellerPct_AMRHT','SellerPct_ALS','SellerPct_CAFULL','SellerPct_CNTL','SellerPct_CITIZ',
                  'SellerPct_53','SellerPct_FIR','SellerPct_FRDOM','SellerPct_GUILD','SellerPct_CHASE',
                  'SellerPct_LLSL','SellerPct_MATRX','SellerPct_NCM','SellerPct_NATIONST','SellerPct_NRESM',
                  'SellerPct_PNYMAC','SellerPct_PILOSI','SellerPct_QUICK','SellerPct_REG','SellerPct_RMSC',
                  'SellerPct_UNSHFI','SellerPct_WFHM']

In [60]:
seller_df[seller_columns] = seller_df[seller_columns].applymap(lambda x: float(x) if isfloat(x) else np.nan)

In [61]:
seller_df.fillna(0,inplace=True)

In [62]:
seller_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11123306 entries, 1482 to 4436
Data columns (total 25 columns):
 #   Column              Non-Null Count     Dtype  
---  ------              --------------     -----  
 0   poolno              11123306 non-null  object 
 1   asofdate            11123306 non-null  int64  
 2   SellerPct_AMRHT     11123306 non-null  float64
 3   SellerPct_ALS       11123306 non-null  float64
 4   SellerPct_CAFULL    11123306 non-null  float64
 5   SellerPct_CNTL      11123306 non-null  float64
 6   SellerPct_CITIZ     11123306 non-null  float64
 7   SellerPct_53        11123306 non-null  float64
 8   SellerPct_FIR       11123306 non-null  float64
 9   SellerPct_FRDOM     11123306 non-null  float64
 10  SellerPct_GUILD     11123306 non-null  float64
 11  SellerPct_CHASE     11123306 non-null  float64
 12  SellerPct_LLSL      11123306 non-null  float64
 13  SellerPct_MATRX     11123306 non-null  float64
 14  SellerPct_NCM       11123306 non-null  float64
 1

In [67]:
# dill.dump_session('notebook_env_all_3_dfs.db')

# Below we are combining the three DataFrames (attr_df, geo_df, seller_df) into a single DataFrame called df

In [71]:
#dill.load_session('notebook_env_all_3_dfs.db')

In [72]:
print(f'attr_df.shape   = {attr_df.shape}')
print(f'geo_df.shape    = {geo_df.shape}')
print(f'seller_df.shape = {seller_df.shape}')

attr_df.shape   = (12687671, 34)
geo_df.shape    = (11040282, 57)
seller_df.shape = (11123306, 25)


In [73]:
geo_df.drop('poolno',axis=1,inplace=True)

In [74]:
df = pd.merge(attr_df,geo_df,on=['cusip','asofdate'],how='left')

In [75]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12687671 entries, 0 to 12687670
Data columns (total 88 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   poolno             12687671 non-null  object 
 1   asofdate           12687671 non-null  int64  
 2   cusip              12687671 non-null  object 
 3   spread             12687671 non-null  object 
 4   SMM                12687671 non-null  float64
 5   DayCount           12687671 non-null  float64
 6   OBal               12687671 non-null  float64
 7   CBal               12687671 non-null  float64
 8   factor             12687671 non-null  float64
 9   Coupon             12687671 non-null  float64
 10  Wac                12687671 non-null  float64
 11  Wam                12687671 non-null  float64
 12  Age                12687671 non-null  float64
 13  aols               12687671 non-null  float64
 14  waols              12687671 non-null  object 
 15  ONLoans      

In [76]:
seller_df.drop('poolno',axis=1,inplace=True)

In [77]:
df = pd.merge(df,seller_df,on=['cusip','asofdate'],how='left')

In [78]:
df.fillna(0,inplace=True)

In [79]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12954704 entries, 0 to 12954703
Data columns (total 110 columns):
 #   Column              Non-Null Count     Dtype  
---  ------              --------------     -----  
 0   poolno              12954704 non-null  object 
 1   asofdate            12954704 non-null  int64  
 2   cusip               12954704 non-null  object 
 3   spread              12954704 non-null  object 
 4   SMM                 12954704 non-null  float64
 5   DayCount            12954704 non-null  float64
 6   OBal                12954704 non-null  float64
 7   CBal                12954704 non-null  float64
 8   factor              12954704 non-null  float64
 9   Coupon              12954704 non-null  float64
 10  Wac                 12954704 non-null  float64
 11  Wam                 12954704 non-null  float64
 12  Age                 12954704 non-null  float64
 13  aols                12954704 non-null  float64
 14  waols               12954704 non-null  object 


In [88]:
# dill.dump_session('../notebook_env__df.db')

In [82]:
print(f'df.shape = {df.shape}')

df.shape = (12954704, 110)


In [93]:
# Write df into a sequence of small files
print(f'df.shape = {df.shape}')

rows = df.shape[0]

N = rows//30000 + 1

for i in range(N):
    df.iloc[i*30000:(i+1)*30000,].to_csv(f'../30y fixed rate data/fnm/conforming/clean data/pools_part{i+1}.csv',index=False)

df.shape = (12954704, 110)
